In [ ]:
# 03_graph_augmentation_experiment.ipynb
import networkx as nx
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Tạo dữ liệu mô phỏng
docs = [
    {"title": "Python cơ bản", "desc": "Học lập trình Python"},
    {"title": "Python nâng cao", "desc": "Các kỹ thuật OOP và module"},
    {"title": "Java cơ bản", "desc": "Giới thiệu lập trình Java"},
]

# FAISS embedding giả lập
model = SentenceTransformer("all-MiniLM-L6-v2")
vectors = np.array(model.encode([d["desc"] for d in docs])).astype('float32')

# Tạo graph quan hệ giữa sách
G = nx.Graph()
G.add_edge("Python cơ bản", "Python nâng cao", weight=0.9)
G.add_edge("Java cơ bản", "Python cơ bản", weight=0.2)

# Truy vấn
query = "Lập trình Python"
q_vec = model.encode([query])

sim = cosine_similarity(q_vec, vectors)[0]
print("FAISS chỉ:", sim)

# FAISS + Graph (fusion)
alpha = 0.7
enhanced_sim = []
for i, doc in enumerate(docs):
    g_bonus = 0
    for n in G.neighbors(doc["title"]):
        g_bonus += G[doc["title"]][n]["weight"]
    enhanced_sim.append(alpha * sim[i] + (1 - alpha) * g_bonus)

print("FAISS + Graph:", enhanced_sim)
